# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [8]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [9]:
env = UnityEnvironment(file_name="./Banana.app")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [10]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [11]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]
States have length: 37


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [12]:
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

Score: 0.0


When finished, you can close the environment.

In [13]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
from unityagents import UnityEnvironment

# point this to the Banana environment binary
env = UnityEnvironment(file_name="./Banana.app")

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# sanity check
env_info = env.reset(train_mode=True)[brain_name]
state = env_info.vector_observations[0]
print("State size:", len(state))
print("Action size:", brain.vector_action_space_size)

ERROR:root:Exception calling application: Ran out of input
Traceback (most recent call last):
  File "/opt/anaconda3/envs/drlnd_x86/lib/python3.6/site-packages/grpc/_server.py", line 385, in _call_behavior
    return behavior(argument, context), True
  File "/opt/anaconda3/envs/drlnd_x86/lib/python3.6/site-packages/unityagents/rpc_communicator.py", line 26, in Exchange
    return self.child_conn.recv()
  File "/opt/anaconda3/envs/drlnd_x86/lib/python3.6/multiprocessing/connection.py", line 251, in recv
    return _ForkingPickler.loads(buf.getbuffer())
EOFError: Ran out of input
ERROR:root:Exception calling application: could not find MARK
Traceback (most recent call last):
  File "/opt/anaconda3/envs/drlnd_x86/lib/python3.6/site-packages/grpc/_server.py", line 385, in _call_behavior
    return behavior(argument, context), True
  File "/opt/anaconda3/envs/drlnd_x86/lib/python3.6/site-packages/unityagents/rpc_communicator.py", line 26, in Exchange
    return self.child_conn.recv()
  File

In [ ]:
import numpy as np
from collections import deque
from matplotlib import pyplot as plt
import torch

# your Agent and QNetwork imports
from dqn_agent import Agent
from model import QNetwork

# ---- sanity checks (helps catch common issues) ----
try:
    _ = env
    _ = brain_name
except NameError as e:
    raise RuntimeError("Make sure you've created the UnityEnvironment (env) and brain_name before training.") from e

# instantiate agent
agent = Agent(state_size=37, action_size=4, seed=0)

def dqn(n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.1, eps_decay=0.995,
        solve_score=13.0, print_every=100, checkpoint_path="checkpoint.pth"):
    """Deep Q-Learning for the Udacity Banana env."""
    scores = []
    scores_window = deque(maxlen=100)
    eps = eps_start
    best_avg = -np.inf

    for i_episode in range(1, n_episodes + 1):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0]
        score = 0.0

        for t in range(max_t):
            action = int(agent.act(state, eps))                  # epsilon-greedy action
            env_info = env.step(action)[brain_name]              # step env
            next_state = env_info.vector_observations[0]
            reward     = env_info.rewards[0]
            done       = env_info.local_done[0]

            agent.step(state, action, reward, next_state, done)  # learn
            state = next_state
            score += reward
            if done:
                break

        scores_window.append(score)
        scores.append(score)

        # decay epsilon
        eps = max(eps_end, eps_decay * eps)

        # logging
        avg = np.mean(scores_window)
        print(f"\rEpisode {i_episode:4d} | Score: {score:6.2f} | Avg(100): {avg:6.2f} | eps: {eps:6.3f}", end="")
        if i_episode % print_every == 0:
            print(f"\rEpisode {i_episode:4d} | Score: {score:6.2f} | Avg(100): {avg:6.2f} | eps: {eps:6.3f}")

        # save best checkpoint
        if avg > best_avg:
            best_avg = avg
            torch.save(agent.qnetwork_local.state_dict(), checkpoint_path)

        # stop when solved
        if avg >= solve_score and i_episode >= 100:
            print(f"\nEnvironment solved in {i_episode-100} episodes! "
                  f"Average Score: {avg:.2f}")
            break

    # load best checkpoint at the end (handy for eval)
    agent.qnetwork_local.load_state_dict(torch.load(checkpoint_path, map_location="cpu"))
    return scores


def watch_agent(n_episodes=3, max_t=1000, render=True):
    """Run greedy policy episodes to visually inspect performance."""
    total_scores = []
    for i in range(1, n_episodes + 1):
        env_info = env.reset(train_mode=not render)[brain_name]  # render = False -> train_mode=True
        state = env_info.vector_observations[0]
        score = 0.0
        while True:
            action = int(agent.act(state, eps=0.0))              # greedy
            env_info = env.step(action)[brain_name]
            next_state = env_info.vector_observations[0]
            reward     = env_info.rewards[0]
            done       = env_info.local_done[0]
            score += reward
            state = next_state
            if done:
                break
        total_scores.append(score)
        print(f"Eval Episode {i}: Score = {score:.2f}")
    print(f"Average Eval Score over {n_episodes} episodes: {np.mean(total_scores):.2f}")
    return total_scores


# ---- Train ----
scores = dqn()

# ---- Plot training curve ----
plt.figure(figsize=(7,4))
plt.plot(np.arange(len(scores)), scores)
plt.xlabel("Episode #")
plt.ylabel("Score")
plt.title("DQN Training Curve")
plt.tight_layout()
plt.show()

# ---- Watch the trained agent (opens window) ----
# Set render=True to see the Unity window; set to False if you're on a headless session.
watch_agent(n_episodes=5, max_t=1000, render=True)


Episode  100 | Score:   4.00 | Avg(100):   0.22 | eps:  0.606
Episode  200 | Score:  10.00 | Avg(100):   2.50 | eps:  0.367
Episode  273 | Score:   9.00 | Avg(100):   4.77 | eps:  0.255

KeyboardInterrupt: 

In [2]:

from unityagents import UnityEnvironment
import torch
from dqn_agent import Agent



# restart the Unity Banana environment
env = UnityEnvironment(file_name="./Banana.app")  # adjust path if needed

brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# instantiate agent
agent = Agent(state_size=37, action_size=4, seed=0)
torch.save(agent.qnetwork_local.state_dict(), "final_checkpoint.pth")
agent = Agent(state_size=37, action_size=4, seed=0)
agent.qnetwork_local.load_state_dict(torch.load("final_checkpoint.pth"))
watch_agent(n_episodes=5, render=True)


ERROR:root:Exception calling application: Ran out of input
Traceback (most recent call last):
  File "/opt/anaconda3/envs/drlnd_x86/lib/python3.6/site-packages/grpc/_server.py", line 385, in _call_behavior
    return behavior(argument, context), True
  File "/opt/anaconda3/envs/drlnd_x86/lib/python3.6/site-packages/unityagents/rpc_communicator.py", line 26, in Exchange
    return self.child_conn.recv()
  File "/opt/anaconda3/envs/drlnd_x86/lib/python3.6/multiprocessing/connection.py", line 251, in recv
    return _ForkingPickler.loads(buf.getbuffer())
EOFError: Ran out of input


KeyboardInterrupt: 